In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core


In [15]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import os

from collections import defaultdict
import torch
import numpy as np
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist

from langchain_community.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_community.graphs import Neo4jGraph

from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget



from dotenv import load_dotenv

load_dotenv()

import matplotlib.pyplot as plt
import io

In [3]:
vrf_data_path = "../../full_data/vrf_data.xlsx"
seva_data_path = "../../full_data/seva_modified2.xlsx"

vrf_df = pd.read_excel(vrf_data_path)
candidate_info_df = pd.read_excel(seva_data_path)

In [4]:
vrf_jobs_df = vrf_df['/'].dropna()[:10]
corpus_str = " ".join(vrf_jobs_df)
corpus_str = corpus_str.replace("\n", " ")
# corpus_str.count(" ")


In [5]:
docs = [Document(page_content=corpus_str)]

# Step 3: Initialize the RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)

# Step 4: Split the document
documents = text_splitter.split_documents(documents=docs)

In [6]:
llm = OllamaFunctions(model="llama3.1", temperature=0, format="json")

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)

/var/folders/j6/588zcvgj0sdc9tfsz36sdcp40000gn/T/ipykernel_14215/4058995579.py:1: LangChainDeprecationWarning: The class `OllamaFunctions` was deprecated in LangChain 0.0.64 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = OllamaFunctions(model="llama3.1", temperature=0, format="json")


In [9]:
graph_documents

[GraphDocument(nodes=[Node(id='Music Producer', type='Person', properties={}), Node(id='Arranger', type='Person', properties={}), Node(id='Content Writer', type='Person', properties={}), Node(id='Data Entry / Analysis Researcher', type='Person', properties={}), Node(id='Mixing Engineer', type='Person', properties={}), Node(id='Singers / Instrumentalists / Musicians', type='Group', properties={}), Node(id='Translator', type='Person', properties={}), Node(id='Goushala Maintenance Dining/Outside Suppport', type='Organization', properties={})], relationships=[Relationship(source=Node(id='Music Producer', type='Person', properties={}), target=Node(id='Singers / Instrumentalists / Musicians', type='Group', properties={}), type='PRODUCED', properties={}), Relationship(source=Node(id='Arranger', type='Person', properties={}), target=Node(id='Singers / Instrumentalists / Musicians', type='Group', properties={}), type='ARRANGED', properties={}), Relationship(source=Node(id='Content Writer', type

In [12]:
graph = Neo4jGraph()


In [ ]:
graph_documents

[GraphDocument(nodes=[Node(id='Music Producer', type='Person', properties={}), Node(id='Arranger', type='Person', properties={}), Node(id='Content Writer', type='Person', properties={}), Node(id='Data Entry / Analysis Researcher', type='Person', properties={}), Node(id='Mixing Engineer', type='Person', properties={}), Node(id='Singers / Instrumentalists / Musicians', type='Group', properties={}), Node(id='Translator', type='Person', properties={}), Node(id='Goushala Maintenance Dining/Outside Suppport', type='Organization', properties={})], relationships=[Relationship(source=Node(id='Music Producer', type='Person', properties={}), target=Node(id='Singers / Instrumentalists / Musicians', type='Group', properties={}), type='PRODUCED', properties={}), Relationship(source=Node(id='Arranger', type='Person', properties={}), target=Node(id='Singers / Instrumentalists / Musicians', type='Group', properties={}), type='ARRANGED', properties={}), Relationship(source=Node(id='Content Writer', type

In [13]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [19]:
def show_graph():
    uri = os.environ.get("NEO4J_URI")
    auth = (os.environ.get("NEO4J_USERNAME"), os.environ.get("NEO4J_PASSWORD"))
    driver = GraphDatabase.driver(uri, auth=auth)
    session = driver.session()
    widget = GraphWidget(graph=session.run("MATCH (s) - [r:!MENTIONS] -> (t) RETURN s, r, t").graph())
    widget.node_label_mapping = 'id'
    return widget

show_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [18]:
print(os.environ.get("NEO4J_URI"))
print(os.environ.get("NEO4J_USERNAME"))
print(os.environ.get("NEO4J_PASSWORD"))

bolt://localhost:7687
neo4j
your_password
